In [39]:
import os
import sys
from pathlib import Path

import boto3
import pandas as pd
import requests
from mypy_boto3_s3 import S3Client

os.chdir("/Users/mrestrepo/git/_personal/proyecto-de-grado")

bucket = "cow-detect-maia"

key_pattern = "{region}/{flyover}/{file_name}"
local_base = Path("data/patches-512-ol-160-m01/train")
sample_size = 20
API_BASE_URL = "http://localhost:8000"

In [22]:
df = pd.read_csv("data/patches-512-ol-160-m01/train_points_gt.csv")

by_img = (
    df.groupby(["images", "labels"])
    .agg({"x": "count"})
    .rename(columns={"x": "count"})
    .sort_values("count", ascending=False)
)
by_img = by_img[by_img["count"] > 5]
by_img.shape

(325, 1)

,,count
images,labels,
L_09_05_16_DSC10034_95.JPG,2,22
L_09_05_16_DSC10034_96.JPG,2,22
L_07_05_16_DSC00414_33.JPG,2,22
L_11_05_16_DSC01346_104.JPG,2,20
L_11_05_16_DSC01346_103.JPG,2,19
...,...,...
0af7b1ea3a107e511353adbaba10c2e55a0bddf2_132.JPG,6,6
L_07_05_16_DSC00647_130.JPG,1,6
L_09_05_16_DSC10176_86.JPG,2,6


In [44]:
session = boto3.Session(region_name="us-east-1", profile_name="dvc-user")
s3: S3Client = session.client("s3")

In [23]:
by_img.sample(sample_size)

,,count
images,labels,
14dc8fa4f9f69e8cea225aaf69affde7e44766bd_73.JPG,6,9
13e34bbaf0ea2043d3ec31137a2ffbff5768dbe8_119.JPG,6,8
1f2b5a52ee1fb08e508bd9db013802077fa3de8c_33.JPG,6,6
0df8cbe13350f5b20d9d4156577709895c315e85_8.JPG,6,6
06d5ce33b84af693cd25a468c6ede67c44ffab8b_130.JPG,6,11
026ae7f5cddac223d6e364b7c16779b271cb3198_144.JPG,6,6
1a30f61c368dfcb2010da94ab84c0c734376f7fd_36.JPG,6,7
L_11_05_16_DSC01109_76.JPG,2,8
09e1e0ddb23a9c3e5ac62b552a856111b5f11942_119.JPG,6,7


In [58]:
import numpy as np

resp = requests.get(f"{API_BASE_URL}/app-info")
model_name = resp.json()["model_info"]["model_metadata"]["name"]
print(f"{model_name=}")
region = {"HerdNet": "Chad-HN", "FasterRCNNResNetFPN": "Chad-FRC"}[model_name]
print(f"{region=}")
np.random.seed(42)
for flyover in ["2025-06-30", "2025-07-31", "2025-08-31", "2025-09-30", "2025-10-31"]:
    sample = by_img.sample(sample_size).reset_index()
    for _, row in sample.iterrows():
        file_name = row["images"]
        print(f"{flyover=}, {file_name=}")
        key = key_pattern.format(region=region, flyover=flyover, file_name=row["images"])

        s3.upload_file(local_base / file_name, bucket, key)

        payload = {"s3_path": f"s3://{bucket}/{key}"}
        resp = requests.post(f"{API_BASE_URL}/predict", json=payload)
        print(resp.status_code)

model_name='FasterRCNNResNetFPN'
region='Chad-FRC'
flyover='2025-06-30', file_name='079557329c366472fd970ee021e37abe2e44c7ca_34.JPG'
200
flyover='2025-06-30', file_name='S_10_05_16_DSC00472_171.JPG'
200
flyover='2025-06-30', file_name='S_09_05_16_DSC00412_25.JPG'
200
flyover='2025-06-30', file_name='L_09_05_16_DSC10034_76.JPG'
200
flyover='2025-06-30', file_name='146396c157f95fd6b6c113e5118e97546d69a431_72.JPG'
200
flyover='2025-06-30', file_name='0fd5ba32f00698a140e080008c2af81b2ae9fa5c_152.JPG'
200
flyover='2025-06-30', file_name='026ae7f5cddac223d6e364b7c16779b271cb3198_144.JPG'
200
flyover='2025-06-30', file_name='09e1e0ddb23a9c3e5ac62b552a856111b5f11942_119.JPG'
200
flyover='2025-06-30', file_name='L_11_05_16_DSC01386_100.JPG'
200
flyover='2025-06-30', file_name='L_09_05_16_DSC10034_24.JPG'
200
flyover='2025-06-30', file_name='14aa6dc307cfb871cedebf75bf6d7ec6df7a2c00_21.JPG'
200
flyover='2025-06-30', file_name='S_10_05_16_DSC00472_157.JPG'
200
flyover='2025-06-30', file_name='00f0